<a href="https://colab.research.google.com/github/2018147559/Data-analysis-Recommend-System/blob/main/Dart_%EC%A0%84%EC%B2%98%EB%A6%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
cd  /content/drive/Shareddrives/DSL_2021-2_추천시스템/data/주식 trade

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
#!pip install dart-fss

In [ ]:
import dart_fss as dart
api_key="991fee6e5dea7040f6c712e57fe3d198b9fd8608"# api key 변수 설정

In [ ]:
dart.set_api_key(api_key = api_key)
# DART 에 공시된 회사 리스트 불러오기 
corp_list = dart.get_corp_list() 

# 삼성전자 검색 
samsung = corp_list.find_by_corp_name('삼성전자', exactly=True)[0] 
# 2021년부터 연간 연결재무제표 불러오기 
fs = samsung.extract_fs(bgn_de='20200101', end_de = '20200331') 
fs[0]

Output()

Output()

Output()

# 거래 데이터 합치기

In [ ]:
cus_info = pd.read_csv("2_cus_info.csv")
act_info = pd.read_csv("2_act_info.csv")
iem_info = pd.read_csv("2_iem_info.csv")
trd_kr = pd.read_csv("2_trd_kr.csv")
cus_act_info = pd.merge(cus_info, act_info, left_on = 'cus_id', right_on = 'cus_id', how = 'left')
cus_act_trd = pd.merge(cus_act_info, trd_kr, left_on = 'act_id', right_on ='act_id', how = 'inner')
df = pd.merge(cus_act_trd, iem_info, left_on = 'iem_cd', right_on = 'iem_cd', how = 'inner')

In [ ]:
df.columns

## 날짜 데이터 전처리

In [ ]:
df['datetime'] = pd.to_datetime(df['orr_dt'].astype('str'))
df['orr_year'] = df['datetime'].dt.year.astype('str')
df['orr_month'] = df['datetime'].dt.month.astype('str')
df['orr_y_m'] = df['orr_year'] + "-" + df['orr_month']
df[['orr_dt','orr_y_m']].head()

In [ ]:
del df['orr_year']
del df['orr_month']
del df['datetime']

### 함수를 편하게 만들기 위해서 df에 열을 추가하겠습니당.
* 날짜 데이터를 바탕으로 그 거래가 발생했을 때 바탕이 됐을 손익계산서의 발행 날짜
* 기준은 3월로 할게요. 예를 들어서 2020년 3월 이후에 한 거래는 2020년의 재무제표를 참고했을 것이라고 가정하겠습니다. 만약 2019년 10월에 거래했다면 2018년의 재무제표를 참고했을 것이라고 가정.
* 이후에 반기보고서나 분기보고서도 참고할 수 있게 된다면 그 때 추가하겠습니당.

=> 한계점: 1월부터 3월에는 재무제표가 새로 나와서 그것이 사용자의 선택/주가에 영향이 클 거야.. 근데 그거를 고려 못 했어ㅠㅠ 일단 해보고 다르게 고칠 방법이 생각나면 고칠게요

In [ ]:
map(int(),df['orr_y_m'])
df['orr_y_m'] = pd.to_datetime(df['orr_y_m'])
df['based_profit'] = np.where(df['orr_y_m']<='2019-03',2018,2019)
df[['based_profit','orr_y_m']].head(5)

In [ ]:
df.groupby(['iem_krl_nm','based_profit']).head()

# 다트에서 특정 표만 가져오기

In [ ]:
dart.set_api_key(api_key = api_key)
# DART 에 공시된 회사 리스트 불러오기 
corp_list = dart.get_corp_list() 

# 삼성전자 검색 
samsung = corp_list.find_by_corp_name('삼성전자', exactly=True)[0] 
# 2021년부터 연간 연결재무제표 불러오기 
fs = samsung.extract_fs(bgn_de='20210101') 
fs[0]

In [ ]:
samsung = corp_list.find_by_corp_name('삼성전자', exactly=True)[0] 
# 2021년부터 연간 연결재무제표 불러오기 
fs = samsung.extract_fs(bgn_de='20190101', end_de = '20190401', separate = True, report_tp = 'annual')
df_fs = fs['is']
labels_fs = fs.labels['is']
df_fs

# 함수 만들기

In [ ]:
df['수익'] = 1
df['영업이익'] = 2
df['당기순이익'] = 3

In [ ]:
df['iem_krl_nm'][0]

In [ ]:
corp_list[3].corp_name

In [ ]:
df.columns

In [ ]:
def get_dart(company_name, year, corp_list) :
    api_key="991fee6e5dea7040f6c712e57fe3d198b9fd8608"# api key 변수 설정
    dart.set_api_key(api_key = api_key)
    company = corp_list.find_by_corp_name(company_name, exactly=True)
    company = company[0]
    #어쩌피 20200101로 해도 2019 정보가 다 나오니까?!
    fs = company.extract_fs(bgn_de='20200101', end_de = '20200331', separate = True, report_tp = 'annual')
    df_fs = fs['is']
    
    # if year == 2018 :
    #     fs = company.extract_fs(bgn_de='20190101', end_de = '20190331', separate = True, report_tp = 'annual')
    #     df_fs = fs['is']
    # elif year == 2019 :
    #     fs = company.extract_fs(bgn_de='20200101', end_de = '20200331', separate = True, report_tp = 'annual')
    #     df_fs = fs['is']

    return df_fs


In [ ]:
def fill_duplicated(df,df_fs, visited):
    for i in range(len(df)):
        if df.iloc[i, 20] == 2018:
            if df_fs is None:
                pass
            else:
                visited[i] = True
                df.iloc[i, 21] = df_fs[0,7]
                df.iloc[i, 22] = df_fs[4,7]
                df.iloc[i, 23] = df_fs[12,7]
        elif df.iloc[i,20] == 2019:
            if df_fs is None:
                pass
            else:
                visited[i] = True
                df.iloc[i, 21] = df_fs[0,6]
                df.iloc[i, 22] = df_fs[4,6]
                df.iloc[i, 23] = df_fs[12,6]

In [ ]:
def fill_profit(df):
    api_key="991fee6e5dea7040f6c712e57fe3d198b9fd8608"# api key 변수 설정
    dart.set_api_key(api_key = api_key)
  # DART 에 공시된 회사 리스트 불러오기 
    corp_list = dart.get_corp_list()
    #한번이라도 채우면 visit 된거니까 다시는 방문 안되도록 체크할 수 있게 visited라는 false로 df의 length만큼 있는 list 생성
    visited = [False for i in range(len(df))]
    for i in range(len(df)):
        # 방문했으면 다시 들릴필요 없음 => pass
        if(visited[i]) continue
        df_fs = get_dart(df.iloc[i,18].strip() , df.iloc[i, 20], corp_list)
        fill_duplicated(df, df_fs, visited)
        
        # if df.iloc[i, 20] == 2018:
        #     if df_fs is None:
        #         pass
        #     else:
        #         visited[i] = True
        #         df.iloc[i, 21] = df_fs[0,7]
        #         df.iloc[i, 22] = df_fs[4,7]
        #         df.iloc[i, 23] = df_fs[12,7]
        # elif df.iloc[i,20] == 2019:
        #     if df_fs is None:
        #         pass
        #     else:
        #         visited[i] = True
        #         df.iloc[i, 21] = df_fs[0,6]
        #         df.iloc[i, 22] = df_fs[4,6]
        #         df.iloc[i, 23] = df_fs[12,6]

In [ ]:
ss = df.iloc[0,18].strip() 

In [ ]:
ss == '네이처셀'                                                    

In [ ]:
fill_profit(df)